In [1]:
import requests 
from bs4 import BeautifulSoup

In [21]:
import requests
from bs4 import BeautifulSoup
import json

def get_climate_finance_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Lấy tiêu đề (title)
    title = soup.find("h1").text.strip()

    # Lấy tóm tắt (abstract)
    abstract_tag = soup.find("div", itemprop="abstract")
    abstract = abstract_tag.text.strip() if abstract_tag else "No abstract available"

    # Lấy nội dung của các phần có thẻ <h2>
    table_of_contents = soup.find_all("h2")
    content_section = soup.find("div", id="newsBody")
    all_headers_in_content = content_section.find_all("h2") if content_section else []

    content_data = []
    
    for i in range(len(all_headers_in_content) - 1):
        # Tìm các đoạn văn (p) trong mỗi phần giữa các thẻ <h2>
        p_elements = all_headers_in_content[i].find_all_next('p')
        section_content = {"header": all_headers_in_content[i].text.strip(), "paragraphs": []}
        
        for p in p_elements:
            # Kiểm tra nếu thẻ p nằm giữa thẻ h2
            if p.find_previous_sibling('h2') == all_headers_in_content[i]:
                section_content["paragraphs"].append(p.text.strip())

        content_data.append(section_content)

    # Xử lý thẻ h2 cuối cùng
    if all_headers_in_content:
        last_header = all_headers_in_content[-1]
        p_elements = last_header.find_all_next('p')
        last_section = {"header": last_header.text.strip(), "paragraphs": []}
        
        for p in p_elements:
            last_section["paragraphs"].append(p.text.strip())
        
        content_data.append(last_section)

    # Tổ chức kết quả thành một dictionary
    result = {
        "title": title,
        "abstract": abstract,
        "content": content_data
    }

    # Trả về dữ liệu dưới dạng JSON
    return json.dumps(result, indent=4)

# Gọi hàm với URL của trang web cần lấy dữ liệu
url = "https://www.iso.org/climate-change/climate-finance"
data = get_climate_finance_info(url)
print(data)


{
    "title": "Climate finance: The key to a sustainable future",
    "abstract": "Climate change is one of the most pressing issues facing our world today. Rising global temperatures, melting ice caps, extreme weather events\u00a0\u2013 the impacts are being felt across the globe and pose significant challenges to economic growth. Urgent action is needed to mitigate climate change and adapt to its effects.\nReversing the climate crisis requires massive global investments. The annual financial needs for mitigation and adaption measures are set to increase steadily to an average estimate of USD\u00a09 trillion in 2030, eventually soaring to a staggering USD\u00a010 trillion a year by 2050.*\u00a0Currently available levels of climate finance are multiple times below the needed amounts.\u00a0Developing countries, where the effects of climate change are most severely felt, find it particularly challenging to finance the necessary measures.\nClimate finance has been a central element of gl